Make sure you download the 2016-2017 Household LSMS survey data for Malawi from https://microdata.worldbank.org/index.php/catalog/lsms. It should be in `../countries/malawi_2016/LSMS/`

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
BASE_DIR = '..'
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif')
COUNTRY = 'rwanda_2010'

COUNTRY_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY)
LSMS_DIR = os.path.join(COUNTRY_DIR, 'LSMS')

In [3]:
# these vary from one LSMS survey to another
CONSUMPTION_FILE = 'RWHR61FL.DTA'
CONSUMPTION_PH_COL = 'rexpagg' # per household
CONSUMPTION_PC_COL = 'rexpaggpc' # per capita

GEOLOCATION_FILE = 'RWGE61FL.dbf'
LATITUDE_COL = 'lat_modified'
LONGITUDE_COL = 'lon_modified'

# purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
PPP = 207.238

In [4]:
for file in [CONSUMPTION_FILE, GEOLOCATION_FILE]:
    assert os.path.isfile(os.path.join(LSMS_DIR, file)), print(f'Could not find {file}')

Could not find RWHR61FL.DTA


AssertionError: None

In [33]:
data = pd.read_stata(os.path.join(LSMS_DIR, CONSUMPTION_FILE))

In [36]:
data.head()

,hhid,hv000,hv001,hv002,hv003,hv004,hv005,hv006,hv007,hv008,...,sh175_7,sh175_8,sh176_1,sh176_2,sh176_3,sh176_4,sh176_5,sh176_6,sh176_7,sh176_8
0,12115,RW6,121,15,2,121,737365,2,2011,1334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12119,RW6,121,19,1,121,737365,3,2011,1335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12121,RW6,121,21,2,121,737365,2,2011,1334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12116,RW6,121,16,2,121,737365,2,2011,1334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,121 3,RW6,121,3,2,121,737365,2,2011,1334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
'rexpagg' in data.columns

False

In [6]:
df = pd.read_stata(os.path.join(LSMS_DIR, CONSUMPTION_FILE))
df['cons_ph'] = df[CONSUMPTION_PH_COL]
df['cons_pc'] = df[CONSUMPTION_PC_COL]
df['people_in_household'] = (df['cons_ph'] / df['cons_pc'])
df['cons_ph'] = df['cons_ph'] / PPP / 365
df = df[['case_id', 'cons_ph', 'people_in_household']]

df_geo = pd.read_stata(os.path.join(LSMS_DIR, GEOLOCATION_FILE))
df_cords = df_geo[['case_id', 'HHID', LATITUDE_COL, LONGITUDE_COL]]
df_cords.rename(columns={LATITUDE_COL: 'lat', LONGITUDE_COL: 'lon'}, inplace=True)

ValueError: Version of given Stata file is not 104, 105, 108, 111 (Stata 7SE), 113 (Stata 8/9), 114 (Stata 10/11), 115 (Stata 12), 117 (Stata 13), or 118 (Stata 14)

In [6]:
df_combined = pd.merge(df, df_cords, on='case_id')

In [7]:
df_combined.head()

,case_id,cons_ph,people_in_household,HHID,lat,lon
0,301025230225,3.935405,3.0,0001c970eecf473099368557e2080b3e,-14.683761,34.915074
1,210374850204,22.901846,5.0,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591
2,311057710075,6.183800,5.0,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503
3,312048040073,5.111379,5.0,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219
4,311097790117,10.988404,7.0,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629


In [8]:
df_combined.drop(['case_id', 'HHID'], axis=1, inplace=True)

In [9]:
df_combined.dropna(inplace=True) # can't use na values
df_combined.head()

,cons_ph,people_in_household,lat,lon
0,3.935405,3.0,-14.683761,34.915074
1,22.901846,5.0,-14.005029,33.794591
2,6.183800,5.0,-16.826165,35.269503
3,5.111379,5.0,-15.004730,35.163219
4,10.988404,7.0,-17.016698,35.079629


In [10]:
df_combined.shape

(12444, 4)

Jean et al makes a minor mistake in computing per capita consumption. Even though the LSMS documentation says `rexpagg` is per capita, `rexpaggpc` is actually the per capita value. `rexpagg` is the household consumption. We can compute per capita consumption in a cluster by summing up `rexpagg` in a cluster and dividing by the number of people in the cluster (that were surveyed).

In [11]:
df_clusters = df_combined.groupby(['lat', 'lon']).sum().reset_index()
df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['people_in_household']
df_clusters.drop(['cons_ph', 'people_in_household'], axis=1, inplace=True)

In [12]:
df_clusters.head()

,lat,lon,cons_pc
0,-17.095150,35.217213,1.477796
1,-17.092351,35.114643,1.314741
2,-17.016698,35.079629,1.626932
3,-16.977243,35.205706,1.733232
4,-16.956385,35.168967,1.131669


In [13]:
df_clusters.shape

(780, 3)

This shows how we can use the downloaded nightlight data to get the nightlights value at these cluster points.

In [14]:
import geoio
img = geoio.GeoImage(NIGHTLIGHTS_DIR)
# pass lon then lat
xPixel, yPixel = img.proj_to_raster(34.915074, -14.683761)

In [15]:
xPixel, yPixel

(25790.308983159237, 10762.551363048204)

In [16]:
im_array = np.squeeze(img.get_data())
im_array.shape

(16801, 43201)

In [17]:
im_array[int(yPixel),int(xPixel)] # this is the nightlight value at the given coordinate

0

In [18]:
import math

def create_space(lat, lon, s=10):
    """Creates a s km x s km square centered on (lat, lon)"""
    v = (180/math.pi)*(500/6378137)*s # roughly 0.045 for s=10
    return lat - v, lon - v, lat + v, lon + v

In [19]:
cluster_nightlights = []
for i,r in df_clusters.iterrows():
    min_lat, min_lon, max_lat, max_lon = create_space(r.lat, r.lon)
    xminPixel, yminPixel = img.proj_to_raster(min_lon, min_lat)
    xmaxPixel, ymaxPixel = img.proj_to_raster(max_lon, max_lat)
    
    xminPixel, xmaxPixel = min(xminPixel, xmaxPixel), max(xminPixel, xmaxPixel)
    yminPixel, ymaxPixel = min(yminPixel, ymaxPixel), max(yminPixel, ymaxPixel)
    
    xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
    cluster_nightlights.append(im_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())

In [20]:
df_clusters['nightlights'] = cluster_nightlights

In [21]:
df_clusters.head()

,lat,lon,cons_pc,nightlights
0,-17.095150,35.217213,1.477796,0.000000
1,-17.092351,35.114643,1.314741,0.000000
2,-17.016698,35.079629,1.626932,0.000000
3,-16.977243,35.205706,1.733232,0.121212
4,-16.956385,35.168967,1.131669,0.502674


In [22]:
os.makedirs(os.path.join(COUNTRY_DIR, 'processed'), exist_ok=True)

In [23]:
df_clusters.to_csv(os.path.join(COUNTRY_DIR, 'processed/clusters.csv'), index=False)